In [1]:
import numpy as np
import tensorflow as tf
import datetime
import os

In [2]:
class MLP:
    def __init__(self,data_file, log_file, save_file, hidden_node_num):
        #Define instance variable
        self.__data_file = data_file
        self.__log_file = log_file
        self.__save_file = save_file
        self.__log = None

        self.__hidden_node_num = hidden_node_num
        self.__hidden_layer_num = len(self.__hidden_node_num)
        
        self.x_train_data = None
        self.y_train_data = None
        self.x_test_data = None
        self.y_test_data = None
        self.__x_data_len = None
        
        self.__y = None
        self.__x = None
        
        self.__w = []
        self.__b = []
        self.__layer = []
        self.__cost = None
        self.__min_cost = 99999999
        self.__min_acc = 99999999
        
        self.__y_out = None
        self.__train = None
        self.__sess = None
        self.__saver = None
        
        self.__loadData()
        self.__init_model()
        
    def __loadData(self):
        #Read data file
        data_file_name = self.__data_file
        xy = np.genfromtxt(data_file_name, dtype='float32')
        
        #Shuffle data
        np.random.shuffle(xy)
        
        #Data Split into train data and test data
        all_data_num = xy[:,1:-1].shape[0]
        train_data_num = int(all_data_num * 95 /100)
        test_data_num = all_data_num - train_data_num
        self.x_train_data =  xy[:train_data_num,1:-1]
        self.y_train_data =  xy[:train_data_num,-1]
        self.x_test_data =  xy[train_data_num:,1:-1]
        self.y_test_data =  xy[train_data_num:,-1]

        #Transpose Matrix having x_data
        self.x_train_data = self.x_train_data.transpose()
        self.x_test_data = self.x_test_data.transpose()

        #Calculate num of variable
        self.__x_data_len = len(self.x_train_data)
        
    def __init_model(self):
        self.__x = tf.placeholder(dtype=tf.float32)
        self.__y = tf.placeholder(dtype=tf.float32)
        
        # first layer
        self.__w.append(tf.Variable(tf.random_normal([self.__hidden_node_num[0], self.__x_data_len]), name="w0"))
        self.__b.append(tf.Variable(tf.random_normal([self.__hidden_node_num[0],1]), name="b0"))
       
        # add hidden layers (variable number)
        for i in range(1,self.__hidden_layer_num):
            wName = "w" + str(i)
            bName = "b" + str(i)
            self.__w.append(tf.Variable(tf.random_normal([self.__hidden_node_num[i], self.__hidden_node_num[i-1]]), name=wName))
            self.__b.append(tf.Variable(tf.random_normal([self.__hidden_node_num[i],1]), name=bName))
        
        # add final layer
        wName = "w" + str(self.__hidden_layer_num)
        bName = "b" + str(self.__hidden_layer_num)
        self.__w.append(tf.Variable(tf.random_normal([1, self.__hidden_node_num[-1]]), name=wName))
        self.__b.append(tf.Variable(tf.random_normal([1],1), name=bName))
        
        # define model
        self.__layer.append(tf.nn.sigmoid(tf.matmul( self.__w[0],self.__x) + self.__b[0]))
        for i in range(1,self.__hidden_layer_num):
            self.__layer.append(tf.nn.sigmoid(tf.matmul( self.__w[i],self.__layer[i-1]) + self.__b[i]))
        self.__y_out = tf.matmul(self.__w[-1],self.__layer[-1]) + self.__b[-1]
        
        # setup cost function and optimizer
        cost = 0
        learning_rate = 0.001
        if(cost == 0)
            # Least Squares
            self.__cost = tf.reduce_mean(tf.square(self.__y_out-  self.__y))
        elif(cost == 1)
            # L2 Loss
            self.__cost = tf.nn.l2_loss(self.__y_out-self.__y)
            
        if(Optimizer == 0)
            # Gradient Descent
            opt = tf.train.GradientDescentOptimizer(learning_rate)
        elif(Optimizer == 1)
            # & Adam
            opt= tf.train.AdamOptimizer(learning_rate)
        elif(Optimizer == 2)
            # Adagrad
            opt= tf.train.tf.train.AdagradOptimizer(learning_rate)
        elif(Optimizer == 3)
            # Momentum 
            # MomentumOptimizer with 0.9 momentum is very standard and works well. 
            # The drawback is that you have to find yourself the best learning rate.
            opt= tf.train.tf.train.MomentumOptimizer(learning_rate,momentum=0.9)    
        elif(Optimizer == 4)
            # Adadelta
            # If you really want to use Adadelta, use the parameters (learning_rate=1., rho=0.95, epsilon=1e-6). 
            # A bigger epsilon will help at the start, but be prepared to wait a bit longer than with other optimizers to see convergence.
            opt= tf.train.AdadeltaOptimizer(1., 0.95, 1e-6)
        elif(Optimizer == 5)
            # RMSProp
            # The results are less dependent on a good learning rate. This algorithm is very similar to Adadelta
            opt= tf.train.RMSPropOptimizer(learning_rate)
            
            
        self.__train = opt.minimize(self.__cost)
    
    def init_sess(self):
        init = tf.global_variables_initializer()
        self.__sess = tf.Session()
        self.__sess.run(init)
        
        if self.__save_file != None:
            self.__saver = tf.train.Saver()
            
        if self.__log_file != None:
            self.__log = open(self.__log_file,'w')
            self.__log.write("Step\tTraning Cost\tTest Accuracy\n")
            self.__log.close()
        
    def model_train(self):
        self.__sess.run(self.__train,feed_dict={self.__x: self.x_test_data, self.__y: self.y_test_data})
        
    def __model_save(self,cost,acc):
        if self.__save_file != None:
            if self.__min_cost > cost:
                self.__min_cost = cost
                self.__saver.save(self.__sess, self.__save_file, write_meta_graph=False)
            elif self.__min_cost == cost:
                if self.__min_acc > acc:
                    self.__min_acc = acc
                    self.__saver.save(self.__sess, self.__save_file, write_meta_graph=False)

    def log_write(self, num):
        if self.__log_file != None:
            predic = self.__sess.run(self.__y_out,feed_dict={self.__x: self.x_test_data})
            acc = np.mean((self.y_test_data- predic) * 100 / predic)
            ccost = self.__sess.run(self.__cost,feed_dict={self.__x: self.x_test_data,self.__y: self.y_test_data})
            self.__log = open(self.__log_file,'a')
            self.__log.write(str(num) + "\t")
            self.__log.write(str(ccost)+ "\t")
            self.__log.write(str(acc)+ "\n")
            self.__log.close()
            
            self.__model_save(ccost,acc)
        
    def close(self):
        self.__sess.close();

In [3]:
hidden_node_num = [10,10,10]
# hidden_node_num = [10]
train_num = 5
print_inter = 1
alpha_num = 5
start_alpha = 0
end_alpha = 4

In [4]:
nowDatetime = datetime.datetime.now().strftime('%m%d_%H%M')
logDir = "log/" + nowDatetime
saveDir= "model/" + nowDatetime
if not os.path.exists(logDir):
    os.makedirs(logDir)
    
if not os.path.exists(saveDir):
    os.makedirs(saveDir)    

model_array = []

for i in range(start_alpha,end_alpha+1):
    dataPath = "data/data_Alpha_" + str(i) + ".txt"
    logPath = logDir + "/log_" + str(i) +".txt"
    savePath = saveDir  + "/model_" + str(i)
    model_array.append(MLP(dataPath, logPath, savePath,hidden_node_num))

In [5]:
print("Traning Start")

for i in range(alpha_num):
    model_array[i].init_sess()
    
for step in range(train_num):
    for i in range(alpha_num):
        model_array[i].model_train()
    if step%print_inter == 0:
        print("Step : ",step)
        for i in range(alpha_num):
            model_array[i].log_write(step)

for i in range(alpha_num):            
    model_array[i].log_write(step)      
    model_array[i].close()
    
print("Traning Finish")

Traning Start
Step :  0
Step :  1
Step :  2
Step :  3
Step :  4
Traning Finish
Traning Finish
